# Configure chain internals at runtime

- **Runtime Configuration of Chain Internals**:
  - LangChain supports the flexibility to experiment with or offer users multiple operational options by providing two methods for runtime configuration:
    1. **Configurable Fields Method**: This method allows for the configuration of specific fields within a runnable.
    2. **Configurable Alternatives Method**: This approach enables listing out alternative runnables that can be set and adjusted during runtime.

## Configuration fields

### With LLMs

In [1]:
# Configure temparature

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField

model = ChatOpenAI(temperature=0).configurable_fields(
  temperature=ConfigurableField(
    id="llm_temperature",
    name="LLM Temperature",
    description="The temperature of the LLM"
  )
)

In [7]:
model.invoke("pick a random number")

AIMessage(content='7')

In [14]:
model.with_config(configurable={
  "llm_temperature": 0.9
}).invoke("pick a random number")

AIMessage(content='37')

In [15]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

In [16]:
chain.invoke({"x": 0})

AIMessage(content='57')

In [17]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='67')

### With HubRunnables

This is useful to allow for switching of prompts

In [20]:
from langchain.runnables.hub import HubRunnable

In [21]:
prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

In [ ]:
# prompt.invoke({"question": "foo", "context": "bar"})

OUTPUT:

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for
question-answering tasks. Use the following pieces of retrieved context to
answer the question. If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.\nQuestion: foo
\nContext: bar \nAnswer:")])

In [ ]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}

In [ ]:
# ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

## Configurable alternatives

### With LLMs

In [ ]:
from langchain.chat_models import ChatAnthropic, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField

llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

# By default it will call Anthropic
chain.invoke({"topic": "bears"})

# We can use `.with_config(configurable={"llm": "openai"})` to specify an llm to use
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})


# If we use the `default_key` then it uses the default
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "bears"})

### With prompts



In [ ]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

# By default it will write a joke
chain.invoke({"topic": "bears"})

# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

### With prompts and LLMs

In [ ]:
llm = ChatOpenAI(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

### Saving configurations

In [ ]:
openai_poem = chain.with_config(configurable={"llm": "openai"})
openai_poem.invoke({"topic": "bears"})